In [1]:
!pip install -U natasha
!pip install ipymarkup

In [2]:
!pip install stop_words

In [3]:
sents = ['Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры.',
 'Свое заявление он разместил в Twitter.',
 '«Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.',
 'Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат.',
 '11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).',
 'В июле аналогичное решение принял Житомирский областной совет.',
 'В начале месяца с предложением к президенту страны Петру Порошенко вернуть Бандере звание Героя Украины обратились депутаты Верховной Рады.',
 'Парламентарии уверены, что признание Бандеры национальным героем поможет в борьбе с подрывной деятельностью против Украины в информационном поле, а также остановит «распространение мифов, созданных российской пропагандой».',
 'Степан Бандера (1909-1959) был одним из лидеров Организации украинских националистов, выступающей за создание независимого государства на территориях с украиноязычным населением.',
 'В 2010 году в период президентства Виктора Ющенко Бандера был посмертно признан Героем Украины, однако впоследствии это решение было отменено судом.']

In [4]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

sg = Segmenter()
mv = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

ne = NamesExtractor(mv)
de = DatesExtractor(mv)
me = MoneyExtractor(mv)
ae = AddrExtractor(mv)

from stop_words import get_stop_words
import re
from ipymarkup import show_span_line_markup

ru_stopwords = get_stop_words('ru')

In [5]:
text = sents[0]
ents = list(ne(text)) + list(ae(text))

In [6]:
ents[0]

Match(
    start=0,
    stop=13,
    fact=Name(
        first='Израиля',
        last='Посол',
        middle=None
    )
)

In [7]:
prepare_ents = []

for ent in ents:
    prepare_ents.append((
    ent.start,
    ent.stop,
    ent.fact.__class__.__name__
    ))

In [8]:
from ipymarkup import show_span_line_markup
show_span_line_markup(
    text,
    prepare_ents
)

# 1. Отсмотрев результаты извлечения, написать правила фильтрации ложно-положительных сущностей - 1.5 балла

In [9]:
def get_names(text):
    extracted_ents = [(ent.start,
                       ent.stop,
                       ent.fact.__class__.__name__) for ent in ents]
    
    show_span_line_markup(text, prepare_ents)

In [10]:
get_names(text)

In [11]:
doc = Doc(text) # другой способ (хорошие результаты и не надо самой правила фильтрации писать), нашла тут https://github.com/EGM86/info_natasha/blob/natasha/natasha/some_examples.ipynb
doc

Doc(text='Посол Израиля на Украине Йоэль Лион признался, чт...)

In [12]:
doc.segment(sg)
display(doc)
display(doc.sents[:2])
display(doc.tokens[:5])

Doc(text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...], sents=[...])

[DocSent(stop=218, text='Посол Израиля на Украине Йоэль Лион признался, чт..., tokens=[...])]

[DocToken(stop=5, text='Посол'),
 DocToken(start=6, stop=13, text='Израиля'),
 DocToken(start=14, stop=16, text='на'),
 DocToken(start=17, stop=24, text='Украине'),
 DocToken(start=25, stop=30, text='Йоэль')]

In [13]:
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
display(doc.tokens[:5])

[DocToken(stop=5, text='Посол', id='1_1', head_id='1_7', rel='nsubj', pos='NOUN', feats=<Anim,Nom,Masc,Sing>),
 DocToken(start=6, stop=13, text='Израиля', id='1_2', head_id='1_1', rel='nmod', pos='PROPN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=14, stop=16, text='на', id='1_3', head_id='1_4', rel='case', pos='ADP'),
 DocToken(start=17, stop=24, text='Украине', id='1_4', head_id='1_1', rel='nmod', pos='PROPN', feats=<Inan,Loc,Fem,Sing>),
 DocToken(start=25, stop=30, text='Йоэль', id='1_5', head_id='1_1', rel='appos', pos='PROPN', feats=<Anim,Nom,Masc,Sing>)]

In [14]:
doc.tag_ner(ner_tagger) # тут выделяются PER (персона), LOC (локация), ORG (организация)
display(doc.spans[:5])

[DocSpan(start=6, stop=13, type='LOC', text='Израиля', tokens=[...]),
 DocSpan(start=17, stop=24, type='LOC', text='Украине', tokens=[...]),
 DocSpan(start=25, stop=35, type='PER', text='Йоэль Лион', tokens=[...]),
 DocSpan(start=89, stop=106, type='LOC', text='Львовской области', tokens=[...]),
 DocSpan(start=152, stop=158, type='LOC', text='России', tokens=[...])]

In [15]:
doc.ner.print()

Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав
      LOC────    LOC──── PER───────                                   
 о решении властей Львовской области объявить 2019 год годом лидера 
                   LOC──────────────                                
запрещенной в России Организации украинских националистов (ОУН) 
              LOC─── ORG─────────────────────────────────────── 
Степана Бандеры.
PER──────────── 


In [16]:
# собственно задание
def get_names_filtered(text):
    extracted_ents = []
    for ent in list(ne(text)) + list(ae(text)):
        try:
            token = ent.fact.last if ent.fact.last else ent.fact.first
        except:
            token = ent.fact.value
        
        if token not in ru_stopwords and re.findall('[А-ЯA-Z]{1}\w+', token): # находим слова, начинающиеся с заглавной буквы, удаляем стоп-слова
            extracted_ents.append((ent.start, 
                                   ent.stop, 
                                   ent.fact.__class__.__name__))
    
    show_span_line_markup(text, extracted_ents)

In [17]:
get_names_filtered(text)

# 2. Визуализировать на N примерах до и после (при этом сравнивать результаты должно быть просто и на глаз) - 1 балл

In [18]:
for sent in sents:
    print('ДО: ') 
    get_names(sent)
    print ('- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -') 
    print('ПОСЛЕ: ')
    get_names_filtered(sent)

ДО: 


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ПОСЛЕ: 


ДО: 


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ПОСЛЕ: 


ДО: 


...

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ПОСЛЕ: 


ДО: 


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ПОСЛЕ: 


ДО: 


- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
ПОСЛЕ: 


# 3. Написать функцию сортировки списка сущностей:

*   использовать полный перебор в дальнейшем  - 0 баллов
*   python sort + lambda - 1 балл
*   custom sort - 3 балла

In [19]:
ents = list(ne(sents[5])) + list(ae(sents[5]))

ents_sorted = sorted(ents, key=lambda x: x.fact.last, reverse=True)
ents_sorted

[Match(
     start=56,
     stop=61,
     fact=Name(
         first=None,
         last='совет',
         middle=None
     )
 ),
 Match(
     start=2,
     stop=6,
     fact=Name(
         first=None,
         last='июле',
         middle=None
     )
 ),
 Match(
     start=34,
     stop=45,
     fact=Name(
         first=None,
         last='Житомирский',
         middle=None
     )
 ),
 Match(
     start=0,
     stop=1,
     fact=Name(
         first=None,
         last='В',
         middle=None
     )
 )]

# 4. Написать правила (минимум 2) по извлечению структурированной информации, (например: имя - локация) - 2 балла

In [20]:
def get_new_ents(sent):
    new_ents = re.findall('[А-ЯA-Z]{1}\w+', sent)
    new_ents = [ent for ent in new_ents if ent not in ru_stopwords]
    return new_ents

In [21]:
for sent in sents:
    print('Text:', sent)
    print('NAMES_LOC: ', ' '.join(get_new_ents(sent)))
    print("\n\n")
    print()

Text: Посол Израиля на Украине Йоэль Лион признался, что пришел в шок, узнав о решении властей Львовской области объявить 2019 год годом лидера запрещенной в России Организации украинских националистов (ОУН) Степана Бандеры.
NAMES_LOC:  Посол Израиля Украине Йоэль Лион Львовской России Организации ОУН Степана Бандеры




Text: Свое заявление он разместил в Twitter.
NAMES_LOC:  Свое Twitter




Text: «Я не могу понять, как прославление тех, кто непосредственно принимал участие в ужасных антисемитских преступлениях, помогает бороться с антисемитизмом и ксенофобией.
NAMES_LOC:  




Text: Украина не должна забывать о преступлениях, совершенных против украинских евреев, и никоим образом не отмечать их через почитание их исполнителей», — написал дипломат.
NAMES_LOC:  Украина




Text: 11 декабря Львовский областной совет принял решение провозгласить 2019 год в регионе годом Степана Бандеры в связи с празднованием 110-летия со дня рождения лидера ОУН (Бандера родился 1 января 1909 года).
NAM

# 5. Добавить еще экстраткоры из наташи (минимум +2 экстратора) и дописать правила фильтрации (если нужны) и правила структурирования - 1.5 балла

In [22]:
ents_add = list(ne(text)) + list(ae(text)) + list(de(text)) + list(me(text))
ents[1]

Match(
    start=2,
    stop=6,
    fact=Name(
        first=None,
        last='июле',
        middle=None
    )
)

In [23]:
def get_filt_names_new(text):
    ents = list(ne(text)) + list(ae(text))
    ents_new = list(de(text)) + list(me(text))
    prepare_ents = []

    for ent in ents:
        
        try:
            token = ent.fact.last if ent.fact.last else ent.fact.first
        except:
            token = ent.fact.value

        if token not in ru_stopwords and re.findall('[А-ЯA-Z]{1}\w+', token):
            prepare_ents.append((
              ent.start,
              ent.stop,
              ent.fact.__class__.__name__,
          ))
            
    for ent in ents_new:
                prepare_ents.append((
                            ent.start,
                            ent.stop,
                            ent.fact.__class__.__name__
                          ))

    
    show_span_line_markup(
                                text,
                                prepare_ents
                                        )

In [24]:
get_filt_names_new(text)

## 6. Сложность алгоритмов

get_names, get_filt_names, get_filt_names_new: О(n) - линейная сложность;

sorted: O(n^2) - квадратичная сложность

Сложность всего пайплайна в целом: О(n^2)

## Задание
1. Отсмотрев результаты извлечения, написать правила фильтрации ложно-положительных сущностей - 1.5 балла

2. Визуализировать на N примерах до и после (при этом сравнивать результаты должно быть просто и на глаз) - 1 балл

3. Написать функцию сортировки списка сущностей:

*   использовать полный перебор в дальнейшем  - 0 баллов
*   python sort + lambda - 1 балл
*   custom sort - 3 балла

4. Написать правила (минимум 2) по извлечению структурированной информации, (например: имя - локация) - 2 балла

5. Добавить еще экстраткоры из наташи (минимум +2 экстратора) и дописать правила фильтрации (если нужны) и правила структурирования - 1.5 балла

6. Оценить по сложности, написанные алгоритмы и весь пайплайн в целом - 1 балл

7. Ускорить и показать ускорение - 2 балла


